In [1]:
#gym imports
import gym
import gym_anytrading

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.agents.dqn import DQNAgent
from rl.agents.ddpg import DDPGAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy,MaxBoltzmannQPolicy,LinearAnnealedPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.random import OrnsteinUhlenbeckProcess
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, TrainEpisodeLogger

#import misc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn import preprocessing
from evaluator import Evaluator

Using TensorFlow backend.


In [2]:
def env_creator():
    return gym.make('forex-v0', frame_bound=(50, 2500), window_size=10)
    env.reset()
    env = env_creator()
    nb_actions = env.action_space.n
    env.observation_space.shape

In [11]:
def buildmodelA():
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + (10, 2)))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('linear'))   
    return model

In [12]:
def buildmodelB():
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + (10, 2)))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('linear'))   
    return model

In [13]:
class EnvProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 2  # (height, width, channel)
        return self.process_obs_1(observation)
    
    def process_obs_1(self, observation):
        prices = []
        diff = []
        for o in observation:
            prices.append(o[0])
            diff.append(o[1])
        prices = preprocessing.normalize([prices], norm='l2', axis=1, copy=True, return_norm=False)[0]
        diff = preprocessing.normalize([diff], norm='l2', axis=1, copy=True, return_norm=False)[0]
        new_obs = np.column_stack((prices, diff))
        return new_obs
    
    def process_obs_2(self, observation):
        prices = []
        diff = []
        for o in observation:
            prices.append(o[0])
            diff.append(o[1] / o[0] * 100)
        prices = preprocessing.normalize([prices], norm='l2', axis=1, copy=True, return_norm=False)[0]
        new_obs = np.column_stack((prices, diff))
        return new_obs
        
    def process_state_batch(self, batch):
#         print(batch)
        return batch

    def process_reward(self, reward):
        return reward

In [14]:
parameter_set="02"
PROCESSOR = EnvProcessor()
#NN
LEARNING_RATE = 0.01
##Policy
#POLICY = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
#POLICY =MaxBoltzmannQPolicy()
POLICY = BoltzmannQPolicy()
#POLICY =CustomEpsGreedyQPolicy(update_interval = 2500, eps = 1.0)
#Memory
MEMORY = SequentialMemory(limit=100000, window_length=1)
# timesteps to observe before training
OBSERVATION = 20 
#Discount Factor
GAMMA = 0.9 # decay rate of past observations original 0.99
#game parameters
TRAIN_INTERVAL = 32
#Size of Minibatch
BATCH_SIZE = 64 

In [15]:
def agent_creator():
    #Model
    MODEL = buildmodelB()
    agent = DQNAgent(model=MODEL, nb_actions=2, memory=MEMORY, nb_steps_warmup=OBSERVATION,
                   target_model_update=1e-2, policy=POLICY, batch_size=BATCH_SIZE, processor=PROCESSOR, 
                   train_interval=TRAIN_INTERVAL, enable_double_dqn=True)
    agent.compile(Adam(lr=LEARNING_RATE), metrics=['mae'])
    return agent

In [ ]:
from evaluator import Evaluator

e = Evaluator(agent_creator, env_creator, name="02")
e.train(repeat=100)

['loss', 'mean_absolute_error', 'mean_q']
starting train 1
------------------ new episode begin, environment reset -----------------------------
Interval 1.0 (0 steps performed)
  1/489 [..............................] - ETA: 36s - reward: 24.3000 - cumulative_reward: 24.3000WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\ProgramData\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


335/489 [===================>..........] - ETA: 0s - reward: -0.6296 - cumulative_reward: -170.1460

C:\ProgramData\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


460/489 [===========================>..] - ETA: 0s - reward: -5.4645 - cumulative_reward: -570.8278
train 1 completed. total_reward: -648.2999999999365 total_profit: 0.8397711780241123
min reward: -170.40000000000165, max reward: 100.40000000000049, mean_reward: -0.2647202939975241

['loss', 'mean_absolute_error', 'mean_q']
starting train 2
------------------ new episode begin, environment reset -----------------------------
Interval 1.0 (0 steps performed)
  1/489 [..............................] - ETA: 1:03 - reward: 24.3000 - cumulative_reward: 24.3000

...

471/489 [===========================>..] - ETA: 0s - reward: 6.8393 - cumulative_reward: -200.1039
train 32 completed. total_reward: -226.2000000000075 total_profit: 0.8333819868364196
min reward: -112.69999999999891, max reward: 168.70000000000164, mean_reward: -0.09236423029808391

['loss', 'mean_absolute_error', 'mean_q']
starting train 33
------------------ new episode begin, environment reset -----------------------------
Interval 1.0 (0 steps performed)


C:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


  1/489 [..............................] - ETA: 18:28 - reward: 0.0000e+00 - cumulative_reward: 0.0000e+00
train 33 completed. total_reward: -84.09999999999584 total_profit: 0.9894102662949491
min reward: -44.500000000000654, max reward: 23.800000000000487, mean_reward: -2.62812499999987

['loss', 'mean_absolute_error', 'mean_q']
starting train 34
------------------ new episode begin, environment reset -----------------------------
Interval 1.0 (0 steps performed)


C:\ProgramData\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


  1/489 [..............................] - ETA: 18:43 - reward: 0.0000e+00 - cumulative_reward: 0.0000e+00